In [7]:
!pip install pandas
!pip install librosa
!pip install shutil
!pip install random
!pip install matplotlib
!pip install --upgrade numpy scipy scikit-learn


ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


  Using cached numpy-2.2.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.2-cp312-cp312-win_amd64.whl (12.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.61.0 requires numpy<2.2,>=1.24, but you have numpy 2.2.2 which is incompatible.


In [8]:
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
import shutil
import random
from sklearn.model_selection import train_test_split

In [26]:
physionet_dirs = [
    rf"C:\Users\SKP\Downloads\classification-of-heart-sound-recordings-the-physionetcomputing-in-cardiology-challenge-2016-1.0.0\classification-of-heart-sound-recordings-the-physionet-computing-in-cardiology-challenge-2016-1.0.0\training-{x}"
    for x in ['a', 'b', 'c', 'd', 'e', 'f']
] + [
    r"C:\Users\SKP\Downloads\classification-of-heart-sound-recordings-the-physionetcomputing-in-cardiology-challenge-2016-1.0.0\classification-of-heart-sound-recordings-the-physionet-computing-in-cardiology-challenge-2016-1.0.0\training",
    r"C:\Users\SKP\Downloads\classification-of-heart-sound-recordings-the-physionetcomputing-in-cardiology-challenge-2016-1.0.0\classification-of-heart-sound-recordings-the-physionet-computing-in-cardiology-challenge-2016-1.0.0\validation"
]

circor_dir = r"C:\Users\SKP\Downloads\the-circor-digiscope-phonocardiogram-dataset-1.0.3 (1)\the-circor-digiscope-phonocardiogram-dataset-1.0.3\training_data"
output_dir = r"C:\Users\SKP\Downloads\processed_heart_sounds"
os.makedirs(output_dir, exist_ok=True)

# Extracting labels from PhysioNet .hea files
def extract_physionet_labels(hea_file):
    label = "unknown"
    with open(hea_file, "r") as f:
        for line in f:
            if line.startswith("#") and "label" in line.lower():
                if "normal" in line.lower():
                    label = "normal"
                elif "abnormal" in line.lower():
                    label = "abnormal"
    return label

# Extracting labels from CIRCOR subject metadata (.txt) files
def extract_circor_labels(subject_file):
    label = "unknown"
    with open(subject_file, "r") as f:
        for line in f:
            if "#Outcome:" in line:
                label = line.split(":")[1].strip()
    return label

# Collecting all files
all_files = []
labeled_files = []
unlabeled_files = []
patient_ids = set()

# Processing PhysioNet dataset
for dir_path in physionet_dirs:
    for file in os.listdir(dir_path):
        if file.endswith(".wav"):
            base_name = os.path.splitext(file)[0]
            hea_file = os.path.join(dir_path, base_name + ".hea")
            if os.path.exists(hea_file):
                label = extract_physionet_labels(hea_file)
                patient_id = base_name.split("_")[0]
                if label == "unknown":
                    unlabeled_files.append((os.path.join(dir_path, file), patient_id, label))
                else:
                    labeled_files.append((os.path.join(dir_path, file), patient_id, label))
                patient_ids.add(patient_id)

# Processing CIRCOR dataset
for file in os.listdir(circor_dir):
    if file.endswith(".txt"):  # Subject metadata file
        patient_id = os.path.splitext(file)[0]
        label = extract_circor_labels(os.path.join(circor_dir, file))
        
        # Find corresponding .wav files
        for wav_file in os.listdir(circor_dir):
            if wav_file.startswith(patient_id) and wav_file.endswith(".wav"):
                if label == "unknown":
                    unlabeled_files.append((os.path.join(circor_dir, wav_file), patient_id, label))
                else:
                    labeled_files.append((os.path.join(circor_dir, wav_file), patient_id, label))
                patient_ids.add(patient_id)

# Converting to DataFrame
df_labeled = pd.DataFrame(labeled_files, columns=["file_path", "patient_id", "label"])
df_unlabeled = pd.DataFrame(unlabeled_files, columns=["file_path", "patient_id", "label"])

# Spliting labeled data into train and validation sets based on patient IDs
train_patients, val_patients = train_test_split(df_labeled["patient_id"].unique(), test_size=0.3, random_state=42)

train_files = df_labeled[df_labeled["patient_id"].isin(train_patients)]
val_files = df_labeled[df_labeled["patient_id"].isin(val_patients)]

# Unlabeled data will go into the test set
test_files = df_unlabeled

# Saved train, validation, and test splits
train_csv = os.path.join(output_dir, "train_data.csv")
val_csv = os.path.join(output_dir, "val_data.csv")
test_csv = os.path.join(output_dir, "test_data.csv")

train_files.to_csv(train_csv, index=False)
val_files.to_csv(val_csv, index=False)
test_files.to_csv(test_csv, index=False)

# Moved files to structured folders
train_output = os.path.join(output_dir, "train")
val_output = os.path.join(output_dir, "val")
test_output = os.path.join(output_dir, "test")
os.makedirs(train_output, exist_ok=True)
os.makedirs(val_output, exist_ok=True)
os.makedirs(test_output, exist_ok=True)

for _, row in train_files.iterrows():
    shutil.copy(row["file_path"], os.path.join(train_output, os.path.basename(row["file_path"])))


for _, row in val_files.iterrows():
    shutil.copy(row["file_path"], os.path.join(val_output, os.path.basename(row["file_path"])))

for _, row in test_files.iterrows():
    shutil.copy(row["file_path"], os.path.join(test_output, os.path.basename(row["file_path"])))

print(f"Processed {len(train_files)} training, {len(val_files)} validation, and {len(test_files)} testing heart sound recordings.")


Processed 2216 training, 947 validation, and 3240 testing heart sound recordings.
